In [ ]:
import sqlite3
import pandas as pd
import requests
from io import BytesIO
import zipfile
from datetime import datetime
import time

In [ ]:
# --- SCHRITT 1: SQLite-Datenbank erstellen ---

def create_database(db_path='weihnachten_schnee.db'):
    """Erstellt die Datenbank mit Star-Schema"""
    print("Erstelle Datenbank...")
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # Dimension: Stationen
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS dim_station (
            station_id INTEGER PRIMARY KEY,
            station_name TEXT,
            bundesland TEXT,
            geo_breite REAL,
            geo_laenge REAL,
            hoehe INTEGER,
            von_datum DATE,
            bis_datum DATE,
            jahre_daten REAL
        )
    ''')
    
    # Dimension: Datum
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS dim_datum (
            datum_id INTEGER PRIMARY KEY,
            datum DATE UNIQUE,
            jahr INTEGER,
            monat INTEGER,
            tag INTEGER,
            weihnachtstag TEXT  -- '24. Dezember', '25. Dezember', '26. Dezember'
        )
    ''')
    
    # Fakten: Schneemessungen
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS fakt_schnee (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            station_id INTEGER,
            datum_id INTEGER,
            schnee_vorhanden INTEGER,  -- 0=nein, 1=ja, NULL=keine Messung
            schneehoehe_cm REAL,       -- Original-Wert falls interessant
            qualitaet INTEGER,         -- QN_4 vom DWD
            FOREIGN KEY (station_id) REFERENCES dim_station(station_id),
            FOREIGN KEY (datum_id) REFERENCES dim_datum(datum_id),
            UNIQUE(station_id, datum_id)  -- Eine Messung pro Station/Tag
        )
    ''')
    
    # Indizes für Performance
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_fakt_station ON fakt_schnee(station_id)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_fakt_datum ON fakt_schnee(datum_id)')
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_datum_jahr ON dim_datum(jahr)')
    
    conn.commit()
    print("✓ Datenbank erstellt")
    return conn

In [ ]:
# --- SCHRITT 2: Stationen laden ---

def load_stations(conn, stations_csv='hexagons_mit_wetterstationen.csv'):
    """Lädt Stationsdaten aus deinem vorherigen Output"""
    print("\nLade Stationen in Datenbank...")
    
    from io import StringIO
    
    KLIMA_URL = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/"
    
    response = requests.get(KLIMA_URL)
    import re
    match = re.search(r'href="([^"]*Beschreibung_Stationen\.txt)"', response.text)
    
    if match:
        full_url = KLIMA_URL + match.group(1)
        stations_response = requests.get(full_url)
        stations_response.encoding = 'latin1'
        text = stations_response.text
        
        # Parse Stationsliste
        lines = text.split('\n')
        header_line = None
        for i, line in enumerate(lines):
            if 'Stations_id' in line and 'von_datum' in line:
                header_line = i
                data_start = i + 2
                break
        
        data = []
        for line in lines[data_start:]:
            if line.strip() and not line.startswith('-'):
                parts = line.split()
                if len(parts) >= 8:
                    data.append(parts[:8])
        
        df = pd.DataFrame(data, columns=[
            'station_id', 'von_datum', 'bis_datum',
            'hoehe', 'geo_breite', 'geo_laenge',
            'station_name', 'bundesland'
        ])
        
        df['station_id'] = df['station_id'].astype(int)
        df['geo_breite'] = df['geo_breite'].astype(float)
        df['geo_laenge'] = df['geo_laenge'].astype(float)
        df['hoehe'] = df['hoehe'].astype(int)
        df['von_datum'] = pd.to_datetime(df['von_datum'], format='%Y%m%d')
        df['bis_datum'] = pd.to_datetime(df['bis_datum'], format='%Y%m%d')
        df['jahre_daten'] = ((df['bis_datum'] - df['von_datum']).dt.days / 365.25).round(1)
        
        # In Datenbank schreiben
        df.to_sql('dim_station', conn, if_exists='replace', index=False)
        
        print(f"✓ {len(df)} Stationen geladen")
        return df

In [ ]:
# --- SCHRITT 3: Datumsdimension füllen ---

def create_date_dimension(conn, start_year=1880, end_year=2024):
    """Erstellt Datumsdimension für alle Weihnachtstage"""
    print("\nErstelle Datumsdimension...")
    
    dates = []
    datum_id = 1
    
    for jahr in range(start_year, end_year + 1):
        for tag, label in [(24, '24. Dezember'), (25, '25. Dezember'), (26, '26. Dezember')]:
            datum = datetime(jahr, 12, tag)
            dates.append({
                'datum_id': datum_id,
                'datum': datum.strftime('%Y-%m-%d'),
                'jahr': jahr,
                'monat': 12,
                'tag': tag,
                'weihnachtstag': label
            })
            datum_id += 1
    
    df_dates = pd.DataFrame(dates)
    df_dates.to_sql('dim_datum', conn, if_exists='replace', index=False)
    
    print(f"✓ {len(df_dates)} Datumseinträge erstellt ({start_year}-{end_year})")

In [ ]:
# --- SCHRITT 4: Schneedaten vom DWD laden ---

def download_snow_data_for_station(station_id, base_url):
    """Lädt Schneehöhen-Daten für eine Station"""
    try:
        # Suche nach der ZIP-Datei für diese Station
        response = requests.get(base_url, timeout=10)
        
        import re
        pattern = f'href="(tageswerte_KL_{station_id:05d}_[0-9]+_[0-9]+_hist\\.zip)"'
        match = re.search(pattern, response.text)
        
        if not match:
            return None
        
        zip_url = base_url + match.group(1)
        
        # Download ZIP
        zip_response = requests.get(zip_url, timeout=30)
        
        # Entpacke und lese Produktdatei
        with zipfile.ZipFile(BytesIO(zip_response.content)) as z:
            # Finde die produkt*.txt Datei
            txt_files = [f for f in z.namelist() if f.startswith('produkt_klima_tag')]
            
            if not txt_files:
                return None
            
            with z.open(txt_files[0]) as f:
                df = pd.read_csv(f, sep=';', skipinitialspace=True)
                
                # Bereinige Spaltennamen (Leerzeichen entfernen)
                df.columns = df.columns.str.strip()
                
                return df
        
    except Exception as e:
        print(f"  ⚠ Fehler bei Station {station_id}: {str(e)[:50]}")
        return None

def load_snow_data(conn, df_stations, limit=None):
    """Lädt Schneehöhen für alle Stationen (nur 24.-26. Dez)"""
    print("\nLade Schneehöhen-Daten vom DWD...")
    
    base_url = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/"
    
    # Datum-Lookup erstellen
    cursor = conn.cursor()
    cursor.execute('SELECT datum_id, datum FROM dim_datum')
    datum_lookup = {row[1]: row[0] for row in cursor.fetchall()}
    
    stations_to_process = df_stations.head(limit) if limit else df_stations
    
    total_records = 0
    failed_stations = []
    
    for idx, station in stations_to_process.iterrows():
        station_id = station['station_id']
        
        if (idx + 1) % 50 == 0:
            print(f"  Fortschritt: {idx + 1}/{len(stations_to_process)} Stationen...")
        
        df_data = download_snow_data_for_station(station_id, base_url)
        
        if df_data is None:
            failed_stations.append(station_id)
            continue
        
        # Konvertiere Datum
        df_data['MESS_DATUM'] = pd.to_datetime(df_data['MESS_DATUM'].astype(str), format='%Y%m%d')
        
        # Filtere nur 24.-26. Dezember
        df_christmas = df_data[
            (df_data['MESS_DATUM'].dt.month == 12) & 
            (df_data['MESS_DATUM'].dt.day.isin([24, 25, 26]))
        ].copy()
        
        if len(df_christmas) == 0:
            continue
        
        # Verarbeite Schneehöhe
        df_christmas['datum_str'] = df_christmas['MESS_DATUM'].dt.strftime('%Y-%m-%d')
        df_christmas['datum_id'] = df_christmas['datum_str'].map(datum_lookup)
        
        # Schneehöhe verarbeiten (-999 = keine Messung)
        df_christmas['SHK_TAG'] = pd.to_numeric(df_christmas['SHK_TAG'], errors='coerce')
        df_christmas['schnee_vorhanden'] = df_christmas['SHK_TAG'].apply(
            lambda x: 1 if x > 0 else (0 if x == 0 else None)
        )
        
        # Bereite Fakten vor
        facts = df_christmas[[
            'STATIONS_ID', 'datum_id', 'schnee_vorhanden', 'SHK_TAG', 'QN_4'
        ]].copy()
        
        facts.columns = ['station_id', 'datum_id', 'schnee_vorhanden', 'schneehoehe_cm', 'qualitaet']
        facts = facts.dropna(subset=['datum_id'])
        
        # In Datenbank schreiben (mit INSERT OR REPLACE für Duplikate)
        cursor = conn.cursor()
        for _, row in facts.iterrows():
            cursor.execute('''
                INSERT OR REPLACE INTO fakt_schnee 
                (station_id, datum_id, schnee_vorhanden, schneehoehe_cm, qualitaet)
                VALUES (?, ?, ?, ?, ?)
            ''', (
                int(row['station_id']), 
                int(row['datum_id']), 
                int(row['schnee_vorhanden']) if pd.notna(row['schnee_vorhanden']) else None,
                float(row['schneehoehe_cm']) if pd.notna(row['schneehoehe_cm']) else None,
                int(row['qualitaet']) if pd.notna(row['qualitaet']) else None
            ))
        total_records += len(facts)
    
    conn.commit()
    
    print(f"\n✓ {total_records} Schneemessungen geladen")
    print(f"✓ {len(failed_stations)} Stationen ohne Daten")
    
    return failed_stations

In [ ]:
# --- SCHRITT 5: Hauptprogramm ---

def main():
    # Datenbank erstellen
    conn = create_database('weihnachten_schnee.db')
    
    # Stationen laden
    df_stations = load_stations(conn)
    
    # Datumsdimension erstellen
    create_date_dimension(conn, start_year=1880, end_year=2024)
    
    failed = load_snow_data(conn, df_stations, limit=None)  # Erst mal nur 10 zum Testen
    
    # Statistiken
    print("\n" + "="*80)
    print("DATENBANK-STATISTIKEN:")
    print("="*80)
    
    cursor = conn.cursor()
    
    cursor.execute('SELECT COUNT(*) FROM dim_station')
    print(f"Stationen: {cursor.fetchone()[0]}")
    
    cursor.execute('SELECT COUNT(*) FROM dim_datum')
    print(f"Datumseinträge: {cursor.fetchone()[0]}")
    
    cursor.execute('SELECT COUNT(*) FROM fakt_schnee')
    print(f"Schneemessungen: {cursor.fetchone()[0]}")
    
    cursor.execute('''
        SELECT 
            SUM(CASE WHEN schnee_vorhanden = 1 THEN 1 ELSE 0 END) as mit_schnee,
            SUM(CASE WHEN schnee_vorhanden = 0 THEN 1 ELSE 0 END) as ohne_schnee,
            SUM(CASE WHEN schnee_vorhanden IS NULL THEN 1 ELSE 0 END) as keine_messung
        FROM fakt_schnee
    ''')
    
    stats = cursor.fetchone()
    print(f"\nMit Schnee: {stats[0]}")
    print(f"Ohne Schnee: {stats[1]}")
    print(f"Keine Messung: {stats[2]}")
    
    
    df_example = pd.read_sql_query(query, conn)
    print(df_example.to_string(index=False))
    
    conn.close()
    print("\n✓ Fertig! Datenbank: weihnachten_schnee.db")

if __name__ == '__main__':
    main()